In [172]:
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar
import datetime as dt    
import pandas as pd
from workalendar.america import Colombia
import plotly.express as px
import plotly.io as pio
from statsmodels.tsa.seasonal import STL
import numpy as np
#pd.set_option('display.max_rows', None)  # Show all rows

In [173]:
objetoAPI = pydataxm.ReadDB()    

In [174]:
#all = objetoAPI.get_collections()
#all

In [175]:

df =objetoAPI.get_collections("DemaReal") #El método get_collection con argumentos retorna los cruces de las varibles que se quieren consultar
#df


In [176]:
df_demanda = objetoAPI.request_data(
    "DemaReal",            # Metric name as it appears in metricId
    "Sistema",             # Entity name as it appears in Entity
    dt.date(2010, 1, 4),  # Start date
    dt.date.today()        # End date (no need to wrap today() in another date())
)

In [177]:
df_demanda

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,4599982.49,4388009.35,4321231.74,4323292.23,4429150.08,4708739.05,4853595.95,5266266.89,...,6503998.12,6389679.39,6466148.87,7745333.51,8103922.75,7738531.43,7051390.11,6173813.17,5493623.80,2010-01-04
1,Sistema,Sistema,5095385.61,4843900.96,4753491.51,4710897.11,4869749.59,5128953.20,5303135.20,5684183.18,...,6690612.41,6632619.77,6687450.39,7895210.40,8214594.14,7818785.49,7130676.82,6243279.36,5520233.97,2010-01-05
2,Sistema,Sistema,5149655.42,4932424.32,4808857.56,4739232.21,4844909.17,5128799.81,5327570.42,5764268.31,...,6737647.01,6661447.03,6678017.37,7830745.24,8076738.83,7698084.62,7078288.32,6243169.78,5631130.15,2010-01-06
3,Sistema,Sistema,5061916.32,4853011.49,4744778.56,4744280.10,4862556.10,5128145.76,5286927.59,5708348.98,...,6769794.64,6692939.72,6754685.85,7893024.84,8069937.60,7643394.40,7046247.14,6285097.27,5580318.45,2010-01-07
4,Sistema,Sistema,5178332.25,4940002.61,4824728.99,4835530.71,4951093.56,5203327.59,5402893.88,5796180.23,...,6807708.66,6645258.92,6671553.15,7892934.95,8003605.93,7611179.88,7014478.59,6326371.35,5697808.86,2010-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,Sistema,Sistema,8556236.87,8238581.75,8067263.10,7994678.76,8164468.58,8626911.08,8724126.72,9079341.60,...,10313928.38,10184906.48,10031574.78,10634334.28,10788905.68,10565293.78,10050666.24,9359849.84,8784794.24,2025-04-10
5576,Sistema,Sistema,8356836.60,8104170.77,7929604.34,7800941.00,7873781.47,8242535.70,8402107.80,8773087.70,...,10228781.26,10043272.88,9810426.07,10328002.71,10559425.88,10394293.51,9955677.85,9383497.83,8836947.25,2025-04-11
5577,Sistema,Sistema,8393137.48,8068002.37,7851045.58,7702590.85,7758407.79,7828464.99,7967870.62,8404722.27,...,9656350.76,9619323.17,9595427.89,10215708.87,10385479.13,10175869.47,9817347.18,9375394.32,8889101.89,2025-04-12
5578,Sistema,Sistema,1434838.77,1423830.76,1422345.94,1420840.25,1392783.75,1415638.34,1562458.36,1926006.39,...,2365963.68,1981220.74,1673491.70,1748312.31,1687896.97,1642735.34,1583221.44,1500954.98,1419190.05,2025-04-13


In [178]:
print("Procesando df_demanda para crear df_modelo...")

# 1. Transformar df_demanda a formato largo (una fila por hora)
df_melted = pd.melt(
    df_demanda,
    id_vars=['Date'],
    value_vars=[col for col in df_demanda.columns if col.startswith('Values_Hour')],
    var_name='Hora_str',
    value_name='kWh'  # kWh inicialmente float
)

# Extraer número de hora y convertir a entero (1–24)
df_melted['Hora_num_1_24'] = df_melted['Hora_str'].str.extract(r'(\d+)').astype(int)

# Crear columna Datetime exacta (fecha + hora ajustada 0–23)
df_melted['Datetime'] = (
    pd.to_datetime(df_melted['Date'])
    + pd.to_timedelta(df_melted['Hora_num_1_24'] - 1, unit='h')
)

# 2. Calcular días hábiles (Colombia)
cal = Colombia()
df_melted['Dia_habil'] = df_melted['Datetime'].apply(
    lambda x: 1 if (x.weekday() < 5 and cal.is_working_day(x.date())) else 0
)

# 3. DataFrame base con las columnas solicitadas
df_modelo = df_melted[['Datetime', 'kWh', 'Dia_habil']].copy()

# Añadir Mes (1–12)
df_modelo['Mes'] = df_modelo['Datetime'].dt.month

# Añadir Hora (0–23)
df_modelo['Hour'] = df_modelo['Datetime'].dt.hour

# Función para Estación numérica en Medellín
def get_medellin_season_numeric(month):
    if month in [12, 1, 2, 3]:
        return 1  # Seco_1
    elif month in [4, 5, 6]:
        return 2  # Lluvias_1
    elif month in [7, 8]:
        return 3  # Seco_2
    else:
        return 4  # Lluvias_2

# Añadir Estación (numérica)
df_modelo['Season'] = df_modelo['Mes'].apply(get_medellin_season_numeric)

# Reordenar columnas
df_modelo = df_modelo[['Datetime', 'Mes', 'Hour', 'Season', 'Dia_habil', 'kWh']]

# --- REDONDEAR kWh Y CONVERTIR A ENTERO ---
print("\nRedondeando y convirtiendo 'kWh' a tipo entero...")
df_modelo['kWh'] = df_modelo['kWh'].round(0).astype(np.int64)

# 4. Ordenar y resetear índice temporal
df_modelo = df_modelo.sort_values('Datetime').reset_index(drop=True)

# 5. Usar 'Datetime' como índice
df_modelo.set_index('Datetime', inplace=True)
df_modelo.sort_index(inplace=True)  # opcional, asegura orden cronológico

# Mostrar resultado
print("\nDataFrame df_modelo final (DateTimeIndex):")
print(df_modelo.head())
print("\nÍndice del DataFrame:")
print(df_modelo.index)

# Verificar tipos
print("\nTipos de datos de df_modelo:")
print(df_modelo.info())

# Valores únicos de Season
print("\nValores únicos en 'Season':")
print(df_modelo['Season'].unique())

Procesando df_demanda para crear df_modelo...

Redondeando y convirtiendo 'kWh' a tipo entero...

DataFrame df_modelo final (DateTimeIndex):
                     Mes  Hour  Season  Dia_habil      kWh
Datetime                                                  
2010-01-04 00:00:00    1     0       1          1  4599982
2010-01-04 01:00:00    1     1       1          1  4388009
2010-01-04 02:00:00    1     2       1          1  4321232
2010-01-04 03:00:00    1     3       1          1  4323292
2010-01-04 04:00:00    1     4       1          1  4429150

Índice del DataFrame:
DatetimeIndex(['2010-01-04 00:00:00', '2010-01-04 01:00:00',
               '2010-01-04 02:00:00', '2010-01-04 03:00:00',
               '2010-01-04 04:00:00', '2010-01-04 05:00:00',
               '2010-01-04 06:00:00', '2010-01-04 07:00:00',
               '2010-01-04 08:00:00', '2010-01-04 09:00:00',
               ...
               '2025-04-14 14:00:00', '2025-04-14 15:00:00',
               '2025-04-14 16:00:00', 

In [179]:
df_modelo

,Mes,Hour,Season,Dia_habil,kWh
Datetime,,,,,
2010-01-04 00:00:00,1,0,1,1,4599982
2010-01-04 01:00:00,1,1,1,1,4388009
2010-01-04 02:00:00,1,2,1,1,4321232
2010-01-04 03:00:00,1,3,1,1,4323292
2010-01-04 04:00:00,1,4,1,1,4429150
...,...,...,...,...,...
2025-04-14 19:00:00,4,19,2,1,2003095
2025-04-14 20:00:00,4,20,2,1,1995673
2025-04-14 21:00:00,4,21,2,1,1946604


In [180]:
try:
    df_modelo.to_csv("data/df_demanda.csv")
    # index=False: No guarda el índice del DataFrame (0, 1, 2...) en el archivo
    # encoding='utf-8': Codificación estándar compatible con muchos sistemas
    print("Archivo CSV guardado con éxito.")
except Exception as e:
    print(f"Error al guardar el archivo CSV: {e}")

# --- FIN SECCIÓN PARA GUARDAR CSV ---

Archivo CSV guardado con éxito.
